## BLAST Parser elastic BLAST 
blastn -outfmt '7 std qcovs staxid ssciname' -evalue 1.0e-6
### future change will be
blastn -outfmt '7 std qcovs staxid ssciname stitle' -evalue 1.0e-6

In [1]:
import Bio
import pandas as pd 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
import sys
import seaborn as sns
import subprocess

### Run small jobs locally on iebdev machine

In [30]:
#os.system("makeblastdb -in stripped.fsa -parse_seqids -dbtype prot -taxid_map taxmap -out prot_blastdb")
#os.system("blastn -db ITS_eukaryote_sequences -num_threads 6 -query blast_fasta.fsa -outfmt '7 std qcovs staxid ssciname' -out blastout.tsv")
os.system("blastn -db nt -num_threads 6 -query newplant.fsa -outfmt '7 std qcovs staxid ssciname kingdom' -out blastout.tsv")
#os.system("blast_formatter -archive blastout.asn -outfmt '7 std staxid ssciname' -out blastout.tsv")
#os.system("blastp -db nr -num_threads 6 -query stripped.fsa -outfmt '7 std qcovs staxid ssciname' -out blastout.tsv")

0

### Run large jobs by elasticBLAST  update this for blastn

In [3]:
#os.system("makeblastdb -in stripped.fsa -parse_seqids -dbtype prot -taxid_map taxmap -out prot_blastdb")
#os.system("gsutil gs://mcveigh-test/")
os.system("gsutil ls gs://elasticblast-mcveigh-test >& /dev/null || gsutil mb gs://elasticblast-mcveigh-test")
os.system("./submit-and-wait-for-results.sh GCP_ITS_nr.ini 500")
os.system("gunzip -c batch_*.gz > blastout.tsv")
os.system("gsutil rm gs://mcveigh-test/results/batch'*'.gz")

Pending 1
Running 0
Succeeded 0
Failed 0
Pending 1
Running 0
Succeeded 0
Failed 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0


Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0
Failed 0
Pending 0
Pending 0
Running 1
Succeeded 0

CommandException: No URLs matched: gs://mcveigh-test/results/cytb/batch*.gz


256

### Read in blastout.tsv file

In [2]:
file_name_string = (r'blastout.tsv')
table_df = pd.read_csv(file_name_string,
                       sep='\t',
                       index_col=None,
                       low_memory=False,
                       usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14],
                       header=None,
                       names=["queryseqid", "subjectseqid", "identity", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "Evalue", "bitscore", "qcoverage", "taxid", "taxname"])
table_df = table_df[~table_df['queryseqid'].str.contains("#")]
table_df['%identity'] = pd.to_numeric(table_df['identity'])
table_df['length'] = pd.to_numeric(table_df['length'])
table_df['qstart'] = pd.to_numeric(table_df['qstart'])
table_df['qend'] = pd.to_numeric(table_df['qend'])
table_df['sstart'] = pd.to_numeric(table_df['sstart'])
table_df['send'] = pd.to_numeric(table_df['send'])
table_df['qcoverage'] = pd.to_numeric(table_df['qcoverage'])
table_df.drop(['mismatch', 'gapopen', 'bitscore', '%identity'], axis=1, inplace=True)
table2_df = table_df.loc[(table_df['queryseqid']) != (table_df['subjectseqid'])]
table2_df

,queryseqid,subjectseqid,identity,length,qstart,qend,sstart,send,Evalue,qcoverage,taxid,taxname
6,MT085710.1,MT085708.1,87.987,1232.0,1.0,1184.0,1.0,1222.0,0.000000e+00,100.0,2710868.0,Agriosomyces longus
7,MT085710.1,MT085709.1,87.641,1238.0,1.0,1184.0,1.0,1227.0,0.000000e+00,100.0,2710868.0,Agriosomyces longus
8,MT085710.1,MT085666.1,88.056,1214.0,1.0,1184.0,1.0,1204.0,0.000000e+00,100.0,2710865.0,Joblinomyces apicalis
9,MT085710.1,MT085665.1,88.056,1214.0,1.0,1184.0,1.0,1201.0,0.000000e+00,100.0,2710865.0,Joblinomyces apicalis
10,MT085710.1,MT085670.1,87.974,1214.0,1.0,1184.0,1.0,1203.0,0.000000e+00,100.0,2710865.0,Joblinomyces apicalis
...,...,...,...,...,...,...,...,...,...,...,...,...
1172385,JN399201.1,GU325821.1,89.873,158.0,320.0,477.0,388.0,545.0,1.180000e-50,24.0,162062.0,Aegagropila linnaei
1172386,JN399201.1,GU325820.1,89.873,158.0,320.0,477.0,388.0,545.0,1.180000e-50,24.0,162062.0,Aegagropila linnaei
1172387,JN399201.1,GU325819.1,89.873,158.0,320.0,477.0,388.0,545.0,1.180000e-50,24.0,162062.0,Aegagropila linnaei
1172388,JN399201.1,FN377675.1,89.172,157.0,322.0,478.0,366.0,520.0,1.180000e-50,24.0,641477.0,Boodlea sp. 11 FL-2009


## Create a smaller dataframe with just numberical values

In [3]:
dftidy = table2_df.drop(['taxid', 'taxname'], axis=1)
dftidy

,queryseqid,subjectseqid,identity,length,qstart,qend,sstart,send,Evalue,qcoverage
6,MT085710.1,MT085708.1,87.987,1232.0,1.0,1184.0,1.0,1222.0,0.000000e+00,100.0
7,MT085710.1,MT085709.1,87.641,1238.0,1.0,1184.0,1.0,1227.0,0.000000e+00,100.0
8,MT085710.1,MT085666.1,88.056,1214.0,1.0,1184.0,1.0,1204.0,0.000000e+00,100.0
9,MT085710.1,MT085665.1,88.056,1214.0,1.0,1184.0,1.0,1201.0,0.000000e+00,100.0
10,MT085710.1,MT085670.1,87.974,1214.0,1.0,1184.0,1.0,1203.0,0.000000e+00,100.0
...,...,...,...,...,...,...,...,...,...,...
1172385,JN399201.1,GU325821.1,89.873,158.0,320.0,477.0,388.0,545.0,1.180000e-50,24.0
1172386,JN399201.1,GU325820.1,89.873,158.0,320.0,477.0,388.0,545.0,1.180000e-50,24.0
1172387,JN399201.1,GU325819.1,89.873,158.0,320.0,477.0,388.0,545.0,1.180000e-50,24.0
1172388,JN399201.1,FN377675.1,89.172,157.0,322.0,478.0,366.0,520.0,1.180000e-50,24.0


## Groupby and Aggregate the top ten rows of blast results

In [4]:
table3_df = dftidy.groupby('queryseqid').head(5) 
ident_group = table3_df.groupby('queryseqid').agg(['median', 'min', 'max']) 
ident_group

identity                   length                 qstart       \
             median     min      max  median     min     max median  min   
queryseqid                                                                 
AF063019.1   99.708  99.708  100.000   686.0   684.0   686.0    1.0  1.0   
AF222793.1   99.510  99.510   99.510   612.0   612.0   612.0    1.0  1.0   
AF354081.1   95.564  95.039   96.538  1533.0  1531.0  1533.0    1.0  1.0   
AF354083.1   95.272  94.560   95.331  1544.0  1542.0  1548.0    1.0  1.0   
AF354084.1   95.087  95.013   95.938  1544.0  1542.0  1551.0    2.0  2.0   
...             ...     ...      ...     ...     ...     ...    ...  ...   
MZ569582.1   99.676  99.676   99.839   618.0   618.0   620.0    3.0  1.0   
MZ569583.1   99.836  99.836  100.000   609.0   609.0   609.0    1.0  1.0   
MZ710169.1   99.851  94.993  100.000   677.0   630.0   679.0    4.0  4.0   
OK077525.1   99.818  99.818   99.818   549.0   549.0   549.0    1.0  1.0   
OK103842.1   98.427  98.427   98.427   572.0   572.0   572.0    4.0  4.0   

                    qend  ... sstart    send                 Evalue            \
             max  median  ...    max  median     min     max median  min  max   
queryseqid                ...                                                   
AF063019.1   2.0   685.0  ...    1.0   686.0   684.0   686.0    0.0  0.0  0.0   
AF222793.1   1.0   611.0  ...    7.0   618.0   618.0   618.0    0.0  0.0  0.0   
AF354081.1   1.0  1525.0  ...   14.0  1521.0  1519.0  1538.0    0.0  0.0  0.0   
AF354083.1   4.0  1532.0  ...   14.0  1531.0  1519.0  1561.0    0.0  0.0  0.0   
AF354084.1  18.0  1531.0  ...   17.0  1538.0  1531.0  1561.0    0.0  0.0  0.0   
...          ...     ...  ...    ...     ...     ...     ...    ...  ...  ...   
MZ569582.1   3.0   620.0  ...   70.0   686.0   686.0   686.0    0.0  0.0  0.0   
MZ569583.1   1.0   609.0  ...   78.0   686.0   686.0   686.0    0.0  0.0  0.0   
MZ710169.1  37.0   680.0  ...   15.0   682.0   630.0   692.0    0.0  0.0  0.0   
OK077525.1   1.0   548.0  ...  160.0   561.0   550.0   708.0    0.0  0.0  0.0   
OK103842.1   4.0   575.0  ...   59.0   591.0   585.0   628.0    0.0  0.0  0.0   

           qcoverage                
              median    min    max  
queryseqid                          
AF063019.1     100.0   99.0  100.0  
AF222793.1     100.0  100.0  100.0  
AF354081.1     100.0  100.0  100.0  
AF354083.1     100.0   99.0  100.0  
AF354084.1      99.0   99.0   99.0  
...              ...    ...    ...  
MZ569582.1      99.0   99.0  100.0  
MZ569583.1     100.0  100.0  100.0  
MZ710169.1      99.0   92.0   99.0  
OK077525.1     100.0  100.0  100.0  
OK103842.1      99.0   99.0   99.0  

[2299 rows x 24 columns]

In [5]:
ident_group.columns = ident_group.columns.map('_'.join)
ident_group.columns.get_level_values(0)

Index(['identity_median', 'identity_min', 'identity_max', 'length_median',
       'length_min', 'length_max', 'qstart_median', 'qstart_min', 'qstart_max',
       'qend_median', 'qend_min', 'qend_max', 'sstart_median', 'sstart_min',
       'sstart_max', 'send_median', 'send_min', 'send_max', 'Evalue_median',
       'Evalue_min', 'Evalue_max', 'qcoverage_median', 'qcoverage_min',
       'qcoverage_max'],
      dtype='object')

In [6]:
ident_group

,identity_median,identity_min,identity_max,length_median,length_min,length_max,qstart_median,qstart_min,qstart_max,qend_median,...,sstart_max,send_median,send_min,send_max,Evalue_median,Evalue_min,Evalue_max,qcoverage_median,qcoverage_min,qcoverage_max
queryseqid,,,,,,,,,,,,,,,,,,,,,
AF063019.1,99.708,99.708,100.000,686.0,684.0,686.0,1.0,1.0,2.0,685.0,...,1.0,686.0,684.0,686.0,0.0,0.0,0.0,100.0,99.0,100.0
AF222793.1,99.510,99.510,99.510,612.0,612.0,612.0,1.0,1.0,1.0,611.0,...,7.0,618.0,618.0,618.0,0.0,0.0,0.0,100.0,100.0,100.0
AF354081.1,95.564,95.039,96.538,1533.0,1531.0,1533.0,1.0,1.0,1.0,1525.0,...,14.0,1521.0,1519.0,1538.0,0.0,0.0,0.0,100.0,100.0,100.0
AF354083.1,95.272,94.560,95.331,1544.0,1542.0,1548.0,1.0,1.0,4.0,1532.0,...,14.0,1531.0,1519.0,1561.0,0.0,0.0,0.0,100.0,99.0,100.0
AF354084.1,95.087,95.013,95.938,1544.0,1542.0,1551.0,2.0,2.0,18.0,1531.0,...,17.0,1538.0,1531.0,1561.0,0.0,0.0,0.0,99.0,99.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MZ569582.1,99.676,99.676,99.839,618.0,618.0,620.0,3.0,1.0,3.0,620.0,...,70.0,686.0,686.0,686.0,0.0,0.0,0.0,99.0,99.0,100.0
MZ569583.1,99.836,99.836,100.000,609.0,609.0,609.0,1.0,1.0,1.0,609.0,...,78.0,686.0,686.0,686.0,0.0,0.0,0.0,100.0,100.0,100.0
MZ710169.1,99.851,94.993,100.000,677.0,630.0,679.0,4.0,4.0,37.0,680.0,...,15.0,682.0,630.0,692.0,0.0,0.0,0.0,99.0,92.0,99.0


In [ ]:
ident_group.to_csv(r'ident_group_file', sep ='\t', index = True)

## Identify sequences with values below the pass/fail thresholds %identity and query coverage

In [7]:
identity_low = ident_group[ident_group['identity_max'] < 87 ].index.tolist()
identity_low

['HM990165.1',
 'KC590560.1',
 'KC590572.1',
 'KC590577.1',
 'MF070678.1',
 'MF070679.1',
 'MK828399.1',
 'MN725751.1',
 'MW488152.1']

In [9]:
qcov_low = ident_group[ident_group['qcoverage_max'] < 85].index.tolist()
qcov_low

['KC590519.1', 'KC590593.1', 'KC590599.1', 'KY988506.1', 'MK616582.1']

In [10]:
qcov_lowdf = ident_group[ident_group['qcoverage_max'] < 90]
qcov_lowdf

,identity_median,identity_min,identity_max,length_median,length_min,length_max,qstart_median,qstart_min,qstart_max,qend_median,...,sstart_max,send_median,send_min,send_max,Evalue_median,Evalue_min,Evalue_max,qcoverage_median,qcoverage_min,qcoverage_max
queryseqid,,,,,,,,,,,,,,,,,,,,,
DQ068771.1,100.000,99.818,100.000,543.0,543.0,548.0,30.0,30.0,30.0,572.0,...,1.0,543.0,543.0,548.0,0.000000e+00,0.000000e+00,0.000000e+00,87.0,87.0,88.0
KC590519.1,87.572,86.464,88.430,362.0,346.0,363.0,5.0,5.0,21.0,334.0,...,155.0,476.0,475.0,516.0,8.100000e-89,2.480000e-95,2.830000e-88,71.0,68.0,71.0
KC590593.1,97.054,94.000,98.137,644.0,610.0,650.0,8.0,6.0,20.0,651.0,...,24.0,645.0,611.0,667.0,0.000000e+00,0.000000e+00,0.000000e+00,73.0,69.0,74.0
KC590599.1,87.719,87.619,87.981,627.0,624.0,632.0,9.0,6.0,12.0,601.0,...,7.0,632.0,626.0,637.0,0.000000e+00,0.000000e+00,0.000000e+00,84.0,84.0,84.0
KY988506.1,97.238,85.099,100.000,181.0,48.0,302.0,62.0,57.0,301.0,345.0,...,585.0,382.0,344.0,632.0,1.480000e-77,1.690000e-108,5.910000e-13,65.0,65.0,84.0
MH864418.1,97.401,97.268,99.156,731.0,711.0,732.0,109.0,109.0,129.0,839.0,...,2.0,728.0,709.0,729.0,0.000000e+00,0.000000e+00,0.000000e+00,87.0,85.0,87.0
MK616582.1,99.533,99.378,99.689,643.0,643.0,643.0,1.0,1.0,1.0,643.0,...,1.0,643.0,643.0,643.0,0.000000e+00,0.000000e+00,0.000000e+00,81.0,81.0,81.0
MK645808.1,87.918,87.755,89.925,687.0,665.0,692.0,83.0,83.0,97.0,753.0,...,1.0,677.0,658.0,683.0,0.000000e+00,0.000000e+00,0.000000e+00,86.0,84.0,86.0


## Sort fasta file creating two files [1] seqs that pass [2] seqs that fail

In [11]:
from Bio import SeqIO
sequences = [] 
fail_sequence = []
for seq_record in SeqIO.parse("blast_fasta_combined", "fasta"):
    s = seq_record
    if seq_record.id in identity_low:
        print("fails seq identity test", seq_record.id)
        fail_sequence.append(s)
    elif seq_record.id in qcov_low:
        fail_sequence.append(s)
        print("fails qcov test",seq_record.id)
    else:
        sequences.append(s)
        #print(seq_record.id)
        
SeqIO.write(sequences, "newFungalpassing", "fasta")

fails seq identity test MW488152.1
fails seq identity test MK828399.1
fails qcov test MK616582.1
fails seq identity test MF070679.1
fails seq identity test MF070678.1
fails qcov test KY988506.1
fails qcov test KC590599.1
fails qcov test KC590593.1
fails seq identity test KC590577.1
fails seq identity test KC590572.1
fails seq identity test KC590560.1
fails qcov test KC590519.1
fails seq identity test HM990165.1
fails seq identity test MN725751.1


2285

In [12]:
sh = open("blast_fasta_combined")
num = 0
for line in sh:
    if line.startswith(">"):
        num += 1
sh.close()
print("Number of sequences in the starting input file:", num)

Number of sequences in the starting input file: 2299


In [14]:
fh = open("newFungalpassing")
n = 0
for line in fh:
    if line.startswith(">"):
        n += 1
fh.close()
print("Number of sequences passing all validation tests and saved to outputfile:", n)

Number of sequences passing all validation tests and saved to outputfile: 2285
